In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_21160\3609341226.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
import yfinance as yf
tick = yf.Ticker('aapl')

In [3]:
tick.history().iloc[-1].round(2)

Open                 148.03
High                 157.38
Low                  147.83
Close                156.73
Volume          60643914.00
Dividends              0.00
Stock Splits           0.00
Name: 2023-02-03 00:00:00, dtype: float64

In [4]:
df = tick.option_chain("2023-03-17").calls
df = df.set_index("strike")
df.iloc[:,2:5].loc[130:170]

,lastPrice,bid,ask
strike,,,
130.0,27.20,27.05,27.35
135.0,22.54,22.40,22.75
140.0,18.00,17.65,17.95
145.0,13.60,13.70,13.95
150.0,9.77,9.95,10.10
155.0,6.59,6.50,6.60
160.0,4.10,4.20,4.25
165.0,2.42,2.50,2.52
170.0,1.35,1.39,1.42


In [5]:
import numpy as np
import plotly.graph_objects as go

strikes = [140, 150, 160]
prems = [df.loc[x, "lastPrice"] for x in strikes]
underlying = np.linspace(120, 180, 100)
values = [np.maximum(underlying-x, 0) for x in strikes]
rets = [v/p - 1 for v, p in zip(values, prems)]

traces = [
  go.Scatter(
    x = underlying,
    y = ret,
    mode="lines",
    hovertemplate="underlying = ${x:.2f}<br>call return = {y:.1%}<extra></extra",
    name=f"{strike:.0f} strike at {prem:.2f}",
  ) for ret, strike, prem in zip(rets, strikes, prems)
]

fig = go.Figure()
for trace in traces:
  fig.add_trace(trace)
fig.update_layout(
    xaxis_title="Underlying Price",
    yaxis_title="Call Option Returns",
    xaxis_title_font = {"size":20},
    yaxis_title_font =  {"size":20},
    template="plotly_white",
    yaxis_tickformat=".0%",
    height=400,  
    width=800,
    legend=dict(
      x = 0.01,
      y = 0.99
    )
)
fig.show()

In [6]:
df = tick.option_chain("2023-03-17").puts
df = df.set_index("strike")
df.iloc[:,2:5].loc[130:170]

,lastPrice,bid,ask
strike,,,
130.0,0.47,0.45,0.46
135.0,0.73,0.74,0.75
140.0,1.15,1.18,1.19
145.0,2.01,1.98,2.00
150.0,3.15,3.10,3.20
155.0,5.01,4.85,4.95
160.0,7.40,7.35,7.45
165.0,10.70,10.45,10.85
170.0,14.80,14.65,14.95


In [7]:
import numpy as np
import plotly.graph_objects as go

strikes = [140, 150, 160]
prems = [df.loc[x, "lastPrice"] for x in strikes]
underlying = np.linspace(120, 180, 100)
values = [np.maximum(underlying-x, 0) for x in strikes]
rets = [v/p - 1 for v, p in zip(values, prems)]

traces = [
  go.Scatter(
    x = underlying,
    y = ret,
    mode="lines",
    hovertemplate="underlying = ${x:.2f}<br>put return = {y:.1%}<extra></extra",
    name=f"{strike:.0f} strike at {prem:.2f}",
  ) for ret, strike, prem in zip(rets, strikes, prems)
]

fig = go.Figure()
for trace in traces:
  fig.add_trace(trace)
fig.update_layout(
    xaxis_title="Underlying Price",
    yaxis_title="Put Option Returns",
    xaxis_title_font = {"size":20},
    yaxis_title_font =  {"size":20},
    template="plotly_white",
    yaxis_tickformat=".0%",
    height=400,  
    width=800,
    legend=dict(
      x = 0.01,
      y = 0.99
    )
)
fig.show()